Foreword:

Hello, in this notebook, we'll create the necessary information tables for our main analysis. (Seasonal stats, Powerplay table, Middle- Overs table, Death overs table)

-------

Importing the necessary libraries:

In [1251]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows',200)
pd.set_option('display.max_columns',30)

In [1252]:
df = pd.read_csv('IPL_Ball_by_Ball_2022.csv')

Tilak Varma and Navdeep Saini have to be renamed to NT Tilak Varma and NA Saini for data integrity.

In [ ]:
df['bowler'].replace({'Navdeep Saini':'NA Saini','Tilak Varma':'NT Tilak Varma'},inplace=True)

In [1253]:
over_data = pd.crosstab(index=df['bowler'],columns=df['overs'],margins=True)

In [1254]:
pd.cut(x=df['overs'],bins=[-1,5,15,19],labels=['Powerplay','Middle Overs','Death'])

0        Powerplay
1        Powerplay
2        Powerplay
3        Powerplay
4        Powerplay
           ...    
17907        Death
17908        Death
17909        Death
17910        Death
17911        Death
Name: overs, Length: 17912, dtype: category
Categories (3, object): ['Powerplay' < 'Middle Overs' < 'Death']

**Creating a new column Over_Category, dividing the overs into Powerplay, death and Middle overs**

In [1255]:
df.insert(loc=4,column='Over_Category',value=pd.cut(x=df['overs'],bins=[-1,5,15,19],labels=['Powerplay','Middle Overs','Death']))

**Creating Data tables to be used**

In [1256]:
Balls_table = pd.crosstab(index=df[(df['extra_type'] != 'wides') & (df['extra_type'] !='noballs')]['bowler'],
                          columns=df[(df['extra_type'] != 'wides') & (df['extra_type'] !='noballs')]['Over_Category'],
                          margins=False)

In [1257]:
W = df[~((df['isWicketDelivery']!=1) | (df['kind'].str.contains('run out|retired hurt|retired out')))]

Wicket_table = pd.crosstab(values=W['isWicketDelivery'],
                    index=W['bowler'],
            columns=W['Over_Category'],
           aggfunc='count',margins=False)

In [1258]:
df[(df['extra_type'] != 'legbyes')&(df['extra_type'] != 'byes')]['extra_type'].value_counts()

wides      702
noballs     69
Name: extra_type, dtype: int64

In [1259]:
pd.crosstab(index=df[df['extra_type'] !='legbyes']['bowler'],
            columns=df[df['extra_type'] !='legbyes']['Over_Category'],
            values=df[df['extra_type'] !='legbyes']['total_run'],
            aggfunc='sum',margins=False)

Runs_table = pd.crosstab(index=df[(df['extra_type'] != 'legbyes')&(df['extra_type'] != 'byes')]['bowler'],
            columns=df[(df['extra_type'] != 'legbyes')&(df['extra_type'] != 'byes')]['Over_Category'],
            values=df[(df['extra_type'] != 'legbyes')&(df['extra_type'] != 'byes')]['total_run'],
            aggfunc='sum',margins=False)

In [1260]:
Avg_table = Runs_table/Wicket_table
Avg_table = round(Avg_table.replace({np.nan:0,np.inf:0}),2)

In [1261]:
SR_table = Balls_table/Wicket_table
SR_table = round(SR_table.replace({np.nan:0,np.inf:0}),2)

In [1262]:
Econ_table = (Runs_table/Balls_table)*6
Econ_table = round(Econ_table.replace({np.nan:0,np.inf:0}),2)

Creating powerplay, middle overs and death table

In [1263]:
Powerplay_table = pd.DataFrame({'Balls':Balls_table['Powerplay'],'Wickets':Wicket_table['Powerplay'],
           'Econ':Econ_table['Powerplay'],'Avg':Avg_table['Powerplay'],'SR':SR_table['Powerplay']})

In [1264]:
Middle_Overs_table = pd.DataFrame({'Balls':Balls_table['Middle Overs'],'Wickets':Wicket_table['Middle Overs'],
           'Econ':Econ_table['Middle Overs'],'Avg':Avg_table['Middle Overs'],'SR':SR_table['Middle Overs']})

In [1265]:
Death_table = pd.DataFrame({'Balls':Balls_table['Death'],'Wickets':Wicket_table['Death'],
           'Econ':Econ_table['Death'],'Avg':Avg_table['Death'],'SR':SR_table['Death']})

Replacing nulls and converting datatypes

In [1266]:
Powerplay_table.replace(np.nan,0,inplace=True)

In [1267]:
Powerplay_table['Wickets'] = Powerplay_table['Wickets'].astype('int64')

In [1268]:
Middle_Overs_table.replace(np.nan,0,inplace=True)

In [1269]:
Middle_Overs_table['Wickets'] = Middle_Overs_table['Wickets'].astype('int64')

In [1270]:
Death_table.replace(np.nan,0,inplace=True)

In [1271]:
Death_table['Wickets'] = Death_table['Wickets'].astype('int64')

Creating overs columns

In [1272]:
o,b = divmod(Death_table['Balls'],6)

jj = pd.DataFrame({'Overs':pd.Series([str(i)+'.'+str(j) for i,j in tuple(zip(o,b))]),'bowler':Death_table.index}).set_index('bowler')

Death_table.insert(loc=0,value=jj['Overs'],column='Overs')

Death_table[Death_table['Balls'] != 0].sort_values('Econ')

,Overs,Balls,Wickets,Econ,Avg,SR
bowler,,,,,,
V Shankar,0.3,3,0,2.00,0.00,0.00
KH Pandya,1.0,6,2,2.00,1.00,3.00
Navdeep Saini,1.0,6,1,5.00,5.00,6.00
R Ashwin,2.0,12,0,5.50,0.00,0.00
SP Narine,5.0,30,1,6.00,30.00,30.00
Ramandeep Singh,2.0,12,3,6.00,4.00,4.00
M Theekshana,4.0,24,4,6.00,6.00,6.00
Rashid Khan,6.0,36,2,6.83,20.50,18.00
GJ Maxwell,1.0,6,0,7.00,0.00,0.00


In [1273]:
o,b = divmod(Middle_Overs_table['Balls'],6)
pd.Series([str(i)+'.'+str(j) for i,j in tuple(zip(o,b))])

kk = pd.DataFrame({'Overs':pd.Series([str(i)+'.'+str(j) for i,j in tuple(zip(o,b))]),'bowler':Middle_Overs_table.index}).set_index('bowler')

Middle_Overs_table.insert(loc=0,value=kk['Overs'],column='Overs')

Middle_Overs_table

,Overs,Balls,Wickets,Econ,Avg,SR
bowler,,,,,,
A Badoni,2.0,12,2,5.50,5.50,6.00
A Nortje,7.2,44,1,11.05,81.00,44.00
AD Russell,13.0,78,7,9.15,17.00,11.14
AF Milne,0.0,0,0,0.00,0.00,0.00
AJ Tye,4.0,24,0,11.50,0.00,0.00
AK Markram,6.0,36,1,10.67,64.00,36.00
AR Patel,34.0,204,4,7.32,62.25,51.00
AS Joseph,14.0,84,3,7.14,33.33,28.00
AS Roy,4.0,24,1,7.75,31.00,24.00


In [1274]:
o,b = divmod(Powerplay_table['Balls'],6)
pd.Series([str(i)+'.'+str(j) for i,j in tuple(zip(o,b))])

ll = pd.DataFrame({'Overs':pd.Series([str(i)+'.'+str(j) for i,j in tuple(zip(o,b))]),'bowler':Powerplay_table.index}).set_index('bowler')

Powerplay_table.insert(loc=0,value=ll['Overs'],column='Overs')

Powerplay_table

,Overs,Balls,Wickets,Econ,Avg,SR
bowler,,,,,,
A Badoni,0.0,0,0,0.00,0.00,0.00
A Nortje,10.0,60,3,9.30,31.00,20.00
AD Russell,4.0,24,3,7.25,9.67,8.00
AF Milne,2.0,12,0,6.00,0.00,0.00
AJ Tye,4.0,24,0,8.00,0.00,0.00
AK Markram,0.0,0,0,0.00,0.00,0.00
AR Patel,8.0,48,2,7.25,29.00,24.00
AS Joseph,9.0,54,1,10.56,95.00,54.00
AS Roy,3.0,18,0,8.00,0.00,0.00


In [1275]:
pd.crosstab(index = df[df['total_run'] == 0]['bowler'],columns=df[df['total_run'] == 0]['Over_Category'],margins=False)

Over_Category,Powerplay,Middle Overs,Death
bowler,,,
A Badoni,0,3,0
A Nortje,26,12,9
AD Russell,15,21,21
AF Milne,9,0,0
AJ Tye,10,6,5
AK Markram,0,8,0
AR Patel,22,63,0
AS Joseph,26,29,12
AS Roy,6,5,0


In [1276]:
dots_table = pd.crosstab(index = df[df['total_run'] == 0]['bowler'],columns=df[df['total_run'] == 0]['Over_Category'],margins=False)

In [1277]:
df[~(df['extra_type'].isnull())][df[~(df['extra_type'].isnull())]['extra_type'].str.contains('byes|legbyes')]

,ID,innings,overs,ballnumber,Over_Category,batter,bowler,non-striker,extra_type,batsman_run,extras_run,total_run,non_boundary,isWicketDelivery,player_out,kind,fielders_involved,BattingTeam
1,1312200,1,0,2,Powerplay,YBK Jaiswal,Mohammed Shami,JC Buttler,legbyes,0,1,1,0,0,NaN,NaN,NaN,Rajasthan Royals
65,1312200,1,10,6,Middle Overs,D Padikkal,HH Pandya,JC Buttler,legbyes,0,1,1,0,0,NaN,NaN,NaN,Rajasthan Royals
198,1312200,2,12,4,Middle Overs,Shubman Gill,TA Boult,HH Pandya,legbyes,0,1,1,0,0,NaN,NaN,NaN,Gujarat Titans
244,1312199,1,1,6,Powerplay,RM Patidar,M Prasidh Krishna,F du Plessis,legbyes,0,4,4,0,0,NaN,NaN,NaN,Royal Challengers Bangalore
253,1312199,1,3,3,Powerplay,RM Patidar,M Prasidh Krishna,F du Plessis,byes,0,4,4,0,0,NaN,NaN,NaN,Royal Challengers Bangalore
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17309,1304049,1,19,6,Death,KD Karthik,Sandeep Sharma,V Kohli,byes,0,1,1,0,0,NaN,NaN,NaN,Royal Challengers Bangalore
17427,1304049,2,17,6,Death,M Shahrukh Khan,Mohammed Siraj,OF Smith,legbyes,0,1,1,0,0,NaN,NaN,NaN,Punjab Kings
17540,1304048,1,17,2,Death,Ishan Kishan,SN Thakur,TH David,legbyes,0,1,1,0,0,NaN,NaN,NaN,Mumbai Indians
17666,1304048,2,16,8,Death,Lalit Yadav,Basil Thampi,AR Patel,legbyes,0,4,4,0,0,NaN,NaN,NaN,Delhi Capitals


In [1278]:
pd.crosstab(index=df[~(df['extra_type'].isnull())][df[~(df['extra_type'].isnull())]['extra_type'].str.contains('byes|legbyes')]['bowler'],
           columns=df[~(df['extra_type'].isnull())][df[~(df['extra_type'].isnull())]['extra_type'].str.contains('byes|legbyes')]['extra_type'])

extra_type,byes,legbyes
bowler,,
A Badoni,0,1
A Nortje,0,2
AD Russell,1,0
AJ Tye,0,3
AR Patel,0,1
AS Joseph,1,4
Abhishek Sharma,0,1
Akash Deep,0,1
Arshdeep Singh,0,8


In [1279]:
Extras_table = pd.crosstab(index=df[(df['extra_type'] == 'wides')|(df['extra_type'] == 'noballs')]['bowler'],
            columns=df[(df['extra_type'] == 'wides')|(df['extra_type'] == 'noballs')]['Over_Category'],
            values=df[(df['extra_type'] == 'wides')|(df['extra_type'] == 'noballs')]['extras_run'],
            aggfunc='sum',margins=False)

In [1280]:
Extras_table

Over_Category,Powerplay,Middle Overs,Death
bowler,,,
A Nortje,2,6,1
AD Russell,6,2,3
AJ Tye,0,2,1
AR Patel,1,3,0
AS Joseph,3,1,2
Akash Deep,1,0,5
Arshdeep Singh,15,4,2
Avesh Khan,3,4,11
B Kumar,15,0,10


In [1281]:
pd.crosstab(index=df[(df['extra_type'] == 'wides')|(df['extra_type'] == 'noballs')]['bowler'],
            columns=df[(df['extra_type'] == 'wides')|(df['extra_type'] == 'noballs')]['Over_Category'],
            values=df[(df['extra_type'] == 'wides')|(df['extra_type'] == 'noballs')]['extras_run'],
            aggfunc='sum',margins=False)

Over_Category,Powerplay,Middle Overs,Death
bowler,,,
A Nortje,2,6,1
AD Russell,6,2,3
AJ Tye,0,2,1
AR Patel,1,3,0
AS Joseph,3,1,2
Akash Deep,1,0,5
Arshdeep Singh,15,4,2
Avesh Khan,3,4,11
B Kumar,15,0,10


In [1282]:
df[(df['extra_type'] == 'wides')|(df['extra_type'] == 'noballs')]

,ID,innings,overs,ballnumber,Over_Category,batter,bowler,non-striker,extra_type,batsman_run,extras_run,total_run,non_boundary,isWicketDelivery,player_out,kind,fielders_involved,BattingTeam
138,1312200,2,3,1,Powerplay,MS Wade,M Prasidh Krishna,Shubman Gill,wides,0,5,5,0,0,NaN,NaN,NaN,Gujarat Titans
148,1312200,2,4,4,Powerplay,HH Pandya,TA Boult,Shubman Gill,wides,0,1,1,0,0,NaN,NaN,NaN,Gujarat Titans
151,1312200,2,4,7,Powerplay,HH Pandya,TA Boult,Shubman Gill,wides,0,1,1,0,0,NaN,NaN,NaN,Gujarat Titans
223,1312200,2,16,5,Death,DA Miller,M Prasidh Krishna,Shubman Gill,wides,0,1,1,0,0,NaN,NaN,NaN,Gujarat Titans
278,1312199,1,7,4,Middle Overs,RM Patidar,R Ashwin,F du Plessis,wides,0,1,1,0,0,NaN,NaN,NaN,Royal Challengers Bangalore
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17677,1304047,1,0,3,Powerplay,RD Gaikwad,UT Yadav,DP Conway,wides,0,1,1,0,0,NaN,NaN,NaN,Chennai Super Kings
17680,1304047,1,0,6,Powerplay,RV Uthappa,UT Yadav,DP Conway,wides,0,1,1,0,0,NaN,NaN,NaN,Chennai Super Kings
17725,1304047,1,7,6,Middle Overs,RV Uthappa,CV Varun,AT Rayudu,wides,0,1,1,0,1,RV Uthappa,stumped,SP Jackson,Chennai Super Kings
17790,1304047,1,18,4,Death,RA Jadeja,Shivam Mavi,MS Dhoni,wides,0,1,1,0,0,NaN,NaN,NaN,Chennai Super Kings


In [1283]:
df['extra_type'].value_counts()

wides      702
legbyes    243
noballs     69
byes        36
Name: extra_type, dtype: int64

In [1284]:
df[df['batsman_run'] == 4].head()

,ID,innings,overs,ballnumber,Over_Category,batter,bowler,non-striker,extra_type,batsman_run,extras_run,total_run,non_boundary,isWicketDelivery,player_out,kind,fielders_involved,BattingTeam
8,1312200,1,1,3,Powerplay,JC Buttler,Yash Dayal,YBK Jaiswal,NaN,4,0,4,0,0,NaN,NaN,NaN,Rajasthan Royals
14,1312200,1,2,3,Powerplay,YBK Jaiswal,Mohammed Shami,JC Buttler,NaN,4,0,4,0,0,NaN,NaN,NaN,Rajasthan Royals
26,1312200,1,4,3,Powerplay,SV Samson,LH Ferguson,JC Buttler,NaN,4,0,4,0,0,NaN,NaN,NaN,Rajasthan Royals
30,1312200,1,5,1,Powerplay,SV Samson,Rashid Khan,JC Buttler,NaN,4,0,4,0,0,NaN,NaN,NaN,Rajasthan Royals
39,1312200,1,6,4,Middle Overs,JC Buttler,LH Ferguson,SV Samson,NaN,4,0,4,0,0,NaN,NaN,NaN,Rajasthan Royals


In [1285]:
fours_table = pd.crosstab(index=df[df['batsman_run'] == 4]['bowler'],
            columns=df[df['batsman_run'] == 4]['Over_Category'],margins=False)

In [1286]:
sixes_table = pd.crosstab(index=df[df['batsman_run'] == 6]['bowler'],
            columns=df[df['batsman_run'] == 6]['Over_Category'],margins=False)

In [1287]:
Powerplay_table

,Overs,Balls,Wickets,Econ,Avg,SR
bowler,,,,,,
A Badoni,0.0,0,0,0.00,0.00,0.00
A Nortje,10.0,60,3,9.30,31.00,20.00
AD Russell,4.0,24,3,7.25,9.67,8.00
AF Milne,2.0,12,0,6.00,0.00,0.00
AJ Tye,4.0,24,0,8.00,0.00,0.00
AK Markram,0.0,0,0,0.00,0.00,0.00
AR Patel,8.0,48,2,7.25,29.00,24.00
AS Joseph,9.0,54,1,10.56,95.00,54.00
AS Roy,3.0,18,0,8.00,0.00,0.00


In [1288]:
Powerplay_table.insert(loc=2,value=Runs_table['Powerplay'],column='Runs')

In [1289]:
Death_table.insert(loc=2,value=Runs_table['Death'],column='Runs')
Middle_Overs_table.insert(loc=2,value=Runs_table['Middle Overs'],column='Runs')

In [1290]:
Death_table

,Overs,Balls,Runs,Wickets,Econ,Avg,SR
bowler,,,,,,,
A Badoni,0.0,0,0,0,0.00,0.00,0.00
A Nortje,5.0,30,43,5,8.60,8.60,6.00
AD Russell,11.1,67,130,7,11.64,18.57,9.57
AF Milne,0.3,3,7,0,14.00,0.00,0.00
AJ Tye,3.0,18,29,2,9.67,14.50,9.00
AK Markram,0.0,0,0,0,0.00,0.00,0.00
AR Patel,1.0,6,14,0,14.00,0.00,0.00
AS Joseph,7.0,42,69,3,9.86,23.00,14.00
AS Roy,0.0,0,0,0,0.00,0.00,0.00


In [1291]:
Extras_table

Over_Category,Powerplay,Middle Overs,Death
bowler,,,
A Nortje,2,6,1
AD Russell,6,2,3
AJ Tye,0,2,1
AR Patel,1,3,0
AS Joseph,3,1,2
Akash Deep,1,0,5
Arshdeep Singh,15,4,2
Avesh Khan,3,4,11
B Kumar,15,0,10


In [1292]:
Powerplay_table

,Overs,Balls,Runs,Wickets,Econ,Avg,SR
bowler,,,,,,,
A Badoni,0.0,0,0,0,0.00,0.00,0.00
A Nortje,10.0,60,93,3,9.30,31.00,20.00
AD Russell,4.0,24,29,3,7.25,9.67,8.00
AF Milne,2.0,12,12,0,6.00,0.00,0.00
AJ Tye,4.0,24,32,0,8.00,0.00,0.00
AK Markram,0.0,0,0,0,0.00,0.00,0.00
AR Patel,8.0,48,58,2,7.25,29.00,24.00
AS Joseph,9.0,54,95,1,10.56,95.00,54.00
AS Roy,3.0,18,24,0,8.00,0.00,0.00


In [1293]:
Powerplay_table.insert(loc=3,value=Extras_table['Powerplay'],column='Extras')
Middle_Overs_table.insert(loc=3,value=Extras_table['Middle Overs'],column='Extras')
Death_table.insert(loc=3,value=Extras_table['Death'],column='Extras')

In [1294]:
Powerplay_table.insert(loc=8,value=fours_table['Powerplay'],column='4s')
Middle_Overs_table.insert(loc=8,value=fours_table['Middle Overs'],column='4s')
Death_table.insert(loc=8,value=fours_table['Death'],column='4s')

In [1295]:
Powerplay_table.insert(loc=9,value=sixes_table['Powerplay'],column='6s')
Middle_Overs_table.insert(loc=9,value=sixes_table['Middle Overs'],column='6s')
Death_table.insert(loc=9,value=sixes_table['Death'],column='6s')

In [1296]:
Powerplay_table.insert(loc=0,value=df.groupby('bowler')[['ID']].nunique()['ID'],column='I')
Middle_Overs_table.insert(loc=0,value=df.groupby('bowler')[['ID']].nunique()['ID'],column='I')
Death_table.insert(loc=0,value=df.groupby('bowler')[['ID']].nunique()['ID'],column='I')

In [1297]:
Powerplay_table.insert(loc=11,value=dots_table['Powerplay'],column='Dots')
Middle_Overs_table.insert(loc=11,value=dots_table['Middle Overs'],column='Dots')
Death_table.insert(loc=11,value=dots_table['Death'],column='Dots')

In [1298]:
pd.crosstab(index=df[df['batsman_run'] == 6]['bowler'],
            columns=df[df['batsman_run'] == 6]['Over_Category'],margins=False)

Over_Category,Powerplay,Middle Overs,Death
bowler,,,
A Nortje,4,5,1
AD Russell,1,6,5
AF Milne,1,0,0
AJ Tye,0,3,1
AK Markram,0,6,0
AR Patel,4,10,1
AS Joseph,6,4,3
AS Roy,1,2,0
Abhishek Sharma,0,2,0


In [1299]:
Powerplay_table.replace({np.nan:0},inplace=True)
Middle_Overs_table.replace({np.nan:0},inplace=True)
Death_table.replace({np.nan:0},inplace=True)

In [1300]:
Powerplay_table[['I','Extras','4s','6s','Dots']] = Powerplay_table[['I','Extras','4s','6s','Dots']].astype('int64')
Middle_Overs_table[['I','Extras','4s','6s','Dots']] = Middle_Overs_table[['I','Extras','4s','6s','Dots']].astype('int64')
Death_table[['I','Extras','4s','6s','Dots']] = Death_table[['I','Extras','4s','6s','Dots']].astype('int64')

In [1301]:
Powerplay_table['Overs'] = Powerplay_table['Overs'].astype('float64')
Middle_Overs_table['Overs'] = Middle_Overs_table['Overs'].astype('float64')
Death_table['Overs'] = Death_table['Overs'].astype('float64')

In [1302]:
Powerplay_table

,I,Overs,Balls,Runs,Extras,Wickets,Econ,Avg,SR,4s,6s,Dots
bowler,,,,,,,,,,,,
A Badoni,2,0.0,0,0,0,0,0.00,0.00,0.00,0,0,0
A Nortje,6,10.0,60,93,2,3,9.30,31.00,20.00,12,4,26
AD Russell,13,4.0,24,29,6,3,7.25,9.67,8.00,3,1,15
AF Milne,1,2.0,12,12,0,0,6.00,0.00,0.00,1,1,9
AJ Tye,3,4.0,24,32,0,0,8.00,0.00,0.00,6,0,10
AK Markram,4,0.0,0,0,0,0,0.00,0.00,0.00,0,0,0
AR Patel,13,8.0,48,58,1,2,7.25,29.00,24.00,3,4,22
AS Joseph,9,9.0,54,95,3,1,10.56,95.00,54.00,11,6,26
AS Roy,2,3.0,18,24,0,0,8.00,0.00,0.00,2,1,6


In [1303]:
Total_stats  = pd.DataFrame({'I':df.groupby('bowler')[['ID']].nunique()['ID'],
              'O':Powerplay_table['Overs']+Middle_Overs_table['Overs']+Death_table['Overs'],
              'Balls':Powerplay_table['Balls']+Middle_Overs_table['Balls']+Death_table['Balls'],
              'Runs':Powerplay_table['Runs']+Middle_Overs_table['Runs']+Death_table['Runs'],
              'Extras':Powerplay_table['Extras']+Middle_Overs_table['Extras']+Death_table['Extras'],
              'Wickets':Powerplay_table['Wickets']+Middle_Overs_table['Wickets']+Death_table['Wickets'],
              '4s':Powerplay_table['4s']+Middle_Overs_table['4s']+Death_table['4s'],
              '6s':Powerplay_table['6s']+Middle_Overs_table['6s']+Death_table['6s'],
              'Dots':Powerplay_table['Dots']+Middle_Overs_table['Dots']+Death_table['Dots']})

In [1304]:
Total_stats

,I,O,Balls,Runs,Extras,Wickets,4s,6s,Dots
bowler,,,,,,,,,
A Badoni,2,2.0,12,11,0,2,0,0,3
A Nortje,6,22.2,134,217,9,9,21,10,47
AD Russell,13,28.1,169,278,11,17,27,12,57
AF Milne,1,2.3,15,19,0,0,2,1,9
AJ Tye,3,11.0,66,107,3,2,13,4,21
AK Markram,4,6.0,36,64,0,1,2,6,8
AR Patel,13,43.0,258,321,4,6,20,15,85
AS Joseph,9,30.0,180,264,6,7,24,13,67
AS Roy,2,7.0,42,55,0,1,2,3,11


In [1305]:
Total_stats.shape

(125, 9)

In [1306]:
Total_stats['Runs']/Total_stats['Wickets']

bowler
A Badoni              5.500000
A Nortje             24.111111
AD Russell           16.352941
AF Milne                   inf
AJ Tye               53.500000
AK Markram           64.000000
AR Patel             53.500000
AS Joseph            37.714286
AS Roy               55.000000
Abdul Samad                inf
Abhishek Sharma            inf
Akash Deep           41.000000
Aman Hakim Khan            inf
Arshdeep Singh       38.500000
Avesh Khan           23.111111
B Kumar              31.916667
Basil Thampi         30.400000
C Sakariya           28.000000
CJ Jordan            67.500000
CV Varun             55.166667
D Brevis              8.000000
D Pretorius          35.000000
DG Nalkande          29.500000
DJ Bravo             18.687500
DJ Hooda             43.000000
DJ Mitchell                inf
DJ Willey            72.000000
DR Sams              28.461538
FA Allen             46.000000
Fazalhaq Farooqi     55.000000
GJ Maxwell           27.500000
HH Pandya            27.750000
H

In [1307]:
Total_stats['Balls']/Total_stats['Wickets']

bowler
A Badoni              6.000000
A Nortje             14.888889
AD Russell            9.941176
AF Milne                   inf
AJ Tye               33.000000
AK Markram           36.000000
AR Patel             43.000000
AS Joseph            25.714286
AS Roy               42.000000
Abdul Samad                inf
Abhishek Sharma            inf
Akash Deep           22.600000
Aman Hakim Khan            inf
Arshdeep Singh       30.000000
Avesh Khan           15.888889
B Kumar              26.083333
Basil Thampi         19.200000
C Sakariya           22.000000
CJ Jordan            38.500000
CV Varun             39.166667
D Brevis              3.000000
D Pretorius          21.000000
DG Nalkande          15.500000
DJ Bravo             12.875000
DJ Hooda             24.000000
DJ Mitchell                inf
DJ Willey            66.000000
DR Sams              19.384615
FA Allen             24.000000
Fazalhaq Farooqi     36.000000
GJ Maxwell           24.000000
HH Pandya            22.875000
H

In [1308]:
Total_stats['Runs']/Total_stats['O']

bowler
A Badoni              5.500000
A Nortje              9.774775
AD Russell            9.893238
AF Milne              8.260870
AJ Tye                9.727273
AK Markram           10.666667
AR Patel              7.465116
AS Joseph             8.800000
AS Roy                7.857143
Abdul Samad           8.000000
Abhishek Sharma       9.500000
Akash Deep           11.081081
Aman Hakim Khan      13.000000
Arshdeep Singh        7.700000
Avesh Khan            8.776371
B Kumar               7.351248
Basil Thampi          9.500000
C Sakariya            7.636364
CJ Jordan            10.800000
CV Varun              8.465473
D Brevis             26.666667
D Pretorius          10.000000
DG Nalkande          11.568627
DJ Bravo              8.742690
DJ Hooda             10.750000
DJ Mitchell          13.500000
DJ Willey             6.545455
DR Sams               8.809524
FA Allen             11.500000
Fazalhaq Farooqi      9.166667
GJ Maxwell            6.875000
HH Pandya             7.326733
H

In [1309]:
Total_stats.reset_index(inplace=True)

In [1310]:
Total_stats['bowler'].replace({'Navdeep Saini':'NA Saini','Tilak Varma':'NT Tilak Varma'},inplace=True)

In [1311]:
# Total_stats = Total_stats.sort_values('bowler')

In [1312]:
# teams_data = pd.read_excel('NEWCSV.xlsx')

In [1313]:
# teams_data[['Player','Team','Hand','Type','Country']]

In [1314]:
# Total_stats.insert(loc=1,column='Team',value=teams_data['Team'])
# Total_stats.insert(loc=2,column='Hand',value=teams_data['Hand'])
# Total_stats.insert(loc=3,column='Type',value=teams_data['Type'])


In [1315]:
# Total_stats.insert(loc=4,column='Country',value=teams_data['Country'])

In [1316]:
Total_stats.sort_values('bowler')

,bowler,I,O,Balls,Runs,Extras,Wickets,4s,6s,Dots
0,A Badoni,2,2.0,12,11,0,2,0,0,3
1,A Nortje,6,22.2,134,217,9,9,21,10,47
2,AD Russell,13,28.1,169,278,11,17,27,12,57
3,AF Milne,1,2.3,15,19,0,0,2,1,9
4,AJ Tye,3,11.0,66,107,3,2,13,4,21
5,AK Markram,4,6.0,36,64,0,1,2,6,8
6,AR Patel,13,43.0,258,321,4,6,20,15,85
7,AS Joseph,9,30.0,180,264,6,7,24,13,67
8,AS Roy,2,7.0,42,55,0,1,2,3,11
9,Abdul Samad,1,1.0,6,8,0,0,1,0,1
